In [9]:
from PIL import Image, ImageDraw, ImageFont, ImageFilter
import numpy as np

In [4]:
background = Image.open('light_background.png')
background.filter(ImageFilter.BLUR)
howai_logo = Image.open('logo_howai_dark_blue.png')
howai_logo = howai_logo.resize((howai_logo.width // 2, howai_logo.height // 2))
background.paste(howai_logo, (0, background.height - 200), howai_logo)
background.show()

In [7]:
class ImageSlideCreator:
    def __init__(self,
                 background_image_path="light_background.png",
                 logo_path="logo_howai_dark_blue.png"):
        """
        Initialize the slide creator with background and logo images
        
        :param background_image_path: Path to the background image
        :param logo_path: Path to the logo image (optional)
        """
        self.im = Image.open(background_image_path)
        self.width = self.im.width
        self.height = self.im.height
        
        # Optional logo (similar to the original class)
        self.logo = Image.open(logo_path).convert("RGBA") if logo_path else None
        
        # Default positioning for logo (can be customized)
        self.logo_position = (0, self.im.height - 200)


    def _scale_image(self, image):
        """
        Scale the image so that either width or height is 10% smaller than background
        
        :param image: PIL Image to be scaled
        :return: Scaled image
        """
        # Calculate scaling factors
        width_scale = (self.width * 0.9) / image.width
        height_scale = (self.height * 0.9) / image.height
        
        # Choose the smaller scaling factor to ensure the image fits
        scale_factor = min(width_scale, height_scale)
        
        # Calculate new dimensions
        new_width = int(image.width * scale_factor)
        new_height = int(image.height * scale_factor)
        
        # Resize the image
        return image.resize((new_width, new_height), Image.LANCZOS)
    

    def create_slide(self, image_path):
        """
        Create a slide with the given image
        
        :param image_path: Path to the image to be added to the slide
        :return: Processed slide image
        """
        # Apply blur to background (optional, mimicking original class)
        self.im = self.im.filter(ImageFilter.BLUR)
        
        # Open and process the new image
        new_image = Image.open(image_path)
        
        # Scale the image to fit within the slide
        new_image = self._scale_image(new_image)
        
        # Calculate position to center the image
        image_x = (self.width - new_image.width) // 2
        image_y = (self.height - new_image.height) // 2
        
        # Paste the new image onto the background
        self.im.paste(new_image, (image_x, image_y), new_image if new_image.mode == 'RGBA' else None)
        
        # Add logo if it exists (optional)
        if self.logo:
            # Resize logo (similar to original class)
            resized_logo = self.logo.resize((int(self.logo.width * 0.5),
                                             int(self.logo.height * 0.5)))
            
            # Paste logo
            self.im.paste(resized_logo, 
                          self.logo_position, 
                          resized_logo)
        
        return self.im

creator = ImageSlideCreator()
slide = creator.create_slide("Screenshot (4)-3.jpeg")
slide.show()
    


In [30]:
from PIL import Image, ImageFilter, ImageStat
import numpy as np

class ImageSlideCreator:
    def __init__(self,
                 background_image_path="light_background.png",
                 logo_light_path="logo_howai.png",
                 logo_dark_path="logo_howai_dark_blue.png"):
        """
        Initialize the slide creator with background and logo images
        
        :param background_image_path: Path to the background image
        :param logo_light_path: Path to the logo for bright backgrounds
        :param logo_dark_path: Path to the logo for dark backgrounds
        """
        self.im = Image.open(background_image_path)
        self.width = self.im.width
        self.height = self.im.height
        
        # Store both logo paths
        self.logo_light_path = logo_light_path
        self.logo_dark_path = logo_dark_path
        
        # Will be set dynamically based on background brightness
        self.logo = None

    def _analyze_bottom_left_brightness(self, image):
        """
        Analyze the brightness of the bottom left corner of the image
        
        :param image: PIL Image to analyze
        :return: Average brightness (0-255)
        """
        # Define the region (bottom left, 10% of image width and height)
        width, height = image.width, image.height
        crop_width = int(width * 0.1)
        crop_height = int(height * 0.1)
        
        # Crop the bottom left corner
        bottom_left_region = image.crop((0, height - crop_height, crop_width, height))
        
        # Convert to grayscale for brightness analysis
        gray_region = bottom_left_region.convert('L')
        
        # Calculate average brightness
        brightness = ImageStat.Stat(gray_region).mean[0]
        print("brightness",brightness)
        return brightness

    def _select_logo(self, image):
        """
        Select appropriate logo based on bottom left corner brightness
        
        :param image: PIL Image to analyze
        :return: Selected logo image
        """
        # Analyze brightness
        brightness = self._analyze_bottom_left_brightness(image)
        
        # Threshold for determining bright/dark (adjust as needed)
        # Lower values mean darker, higher values mean brighter
        brightness_threshold = 127
        
        # Select and load appropriate logo
        if brightness > brightness_threshold:
            logo_path = self.logo_dark_path
        else:
            logo_path = self.logo_light_path
        
        # Load and convert selected logo
        logo = Image.open(logo_path).convert("RGBA")
        return logo

    def create_slide(self, image_path):
        """
        Create a slide with the given image
        
        :param image_path: Path to the image to be added to the slide
        :return: Processed slide image
        """
        # Apply blur to background (optional)
        self.im = self.im.filter(ImageFilter.BLUR)
        
        # Open and process the new image
        new_image = Image.open(image_path)
        
        # Scale the image to fit within the slide
        new_image = self._scale_image(new_image)
        
        # Calculate position to center the image
        image_x = (self.width - new_image.width) // 2
        image_y = (self.height - new_image.height) // 2
        
        # Paste the new image onto the background
        self.im.paste(new_image, (image_x, image_y), new_image if new_image.mode == 'RGBA' else None)
        
        # Select and add logo
        logo = self._select_logo(new_image)
        
        # Resize logo (similar to original approach)
        resized_logo = logo.resize((int(logo.width * 0.5),
                                    int(logo.height * 0.5)))
        
        # Paste logo to bottom left corner
        self.im.paste(resized_logo, 
                      (0, self.im.height - 200), 
                      resized_logo)
        
        return self.im

    def _scale_image(self, image):
        """
        Scale the image so that either width or height is 10% smaller than background
        
        :param image: PIL Image to be scaled
        :return: Scaled image
        """
        # Calculate scaling factors
        width_scale = (self.width * 0.9) / image.width
        height_scale = (self.height * 0.9) / image.height
        
        # Choose the smaller scaling factor to ensure the image fits
        scale_factor = min(width_scale, height_scale)
        
        # Calculate new dimensions
        new_width = int(image.width * scale_factor)
        new_height = int(image.height * scale_factor)
        
        # Resize the image
        return image.resize((new_width, new_height), Image.LANCZOS)
    
creator = ImageSlideCreator()
slide = creator.create_slide("2zu1.png")
slide.show()

brightness 255.0


In [27]:
# create a 4:3 image
image = Image.new('RGB', (800, 400), 'white')
# save the image
image.save('2zu1.png')

In [ ]:
class HeaderSlideCreator:
    def __init__(self, background_image_path = "light_background.png", logo_path = "logo_howai(2).png"):
        self.im = Image.open(background_image_path)
        self.width = self.im.width
        self.height = self.im.height
        self.logo = Image.open(logo_path).convert("RGBA")
        self.font = "arialbd.ttf"

    def _calculate_text_position(self, text : str, font):
        """Calculate the position of the text to be centered"""
        draw = ImageDraw.Draw(Image.new("RGB", (1, 1)))
        text_bbox = draw.textbbox((0, 0), text, font=font)
        text_width = text_bbox[2] - text_bbox[0]
        text_height = text_bbox[3] - text_bbox[1]
        text_x = (self.width - text_width) // 2
        text_y = (self.height - text_height) // 2
        return text_x, text_y


    def create_slide(self, text : str):
        """Create a slide with the given text"""
        self.im = self.im.filter(ImageFilter.BLUR) # blur the background

        self.logo = self.logo.resize((int(self.logo.width * 0.5), 
                                      int(self.logo.height * 0.5))) # resize the logo
        
        self.im.paste(self.logo, 
                      (0, self.im.height - 200), 
                      self.logo) # paste the logo to the bottom left corner
        
        text_layer = Image.new("RGBA", 
                               (self.im.width, self.im.height), 
                               (0, 0, 0, 0)) # create a transparent layer for the text
        
        draw = ImageDraw.Draw(text_layer)
        font_bd = ImageFont.truetype(self.font, 300)
        draw.text((0, 0), text, font=font_bd, fill=(17, 18, 43, 255))
        # get the width and height of the text
        text_x, text_y = self._calculate_text_position(text, font_bd)

        y_offset = -self.im.height // 30
       
        self.im.paste(text_layer, (text_x, text_y + y_offset), text_layer)
        return self.im
